In [64]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Lambda
from keras.layers import LSTM, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import tensorflow as tf

In [65]:
df = pd.read_csv('../../data/scheme-progs.csv')
df = df.sample(frac=1).reset_index(drop=True)
df = df[df.output != 'E']
df.output = pd.to_numeric(df.output)

In [66]:
37389*.8

29911.2

In [67]:
train_x = df.iloc[:30000,0:1].values[:,0]
train_y = df.iloc[:30000,1].values.reshape((30000, 1))
test_x = df.iloc[30000:,0:1].values[:,0]
test_y = df.iloc[30000:,1].values.reshape((7389, 1))

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(30000,) (30000, 1) (7389,) (7389, 1)


In [68]:
SYMBOLS = ['(', ')', ' ', 'x', 'y', '1', '2', '0', '-', 'λ', '+', '-', '*', '/']
N_SYMBOLS = len(SYMBOLS)
TOKENS = dict((c, i) for i, c in enumerate(SYMBOLS))
MAX_LEN = max(list(map(len, (list(train_x)+list(test_x)))))

def tokenize_string(s):
    ret = np.zeros((MAX_LEN, N_SYMBOLS), dtype=bool)
    for i, char in enumerate(s):
        ret[i, TOKENS[char]] = 1
    return ret

def tokenize(a):
#     ret = np.zeros(len(a), MAX_LEN, N_SYMBOLS)
#     sentences = 
#     for i, sentence in enumerate(s):
#         for t, char in enumerate(sentence):
    return np.array(list(map(lambda s: tokenize_string(s), list(a))))

In [69]:
train_x = tokenize(train_x)
test_x = tokenize(test_x)

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(30000, 35, 14) (30000, 1) (7389, 35, 14) (7389, 1)


In [70]:
model = Sequential()
model.add(LSTM(256, input_shape=(MAX_LEN,N_SYMBOLS)))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['accuracy'])
model.fit(train_x,
          train_y,
          batch_size=500,
          epochs=100,
          validation_split=0.3,
          callbacks=[EarlyStopping(patience=4)])

Train on 21000 samples, validate on 9000 samples
Epoch 1/100
21000/21000 [==============================] - 57s - loss: 3.1696 - acc: 0.2190 - val_loss: 2.7718 - val_acc: 0.2373
Epoch 2/100
21000/21000 [==============================] - 51s - loss: 2.8009 - acc: 0.2326 - val_loss: 2.7936 - val_acc: 0.2402
Epoch 3/100
21000/21000 [==============================] - 51s - loss: 2.6909 - acc: 0.2367 - val_loss: 2.5652 - val_acc: 0.2427
Epoch 4/100
21000/21000 [==============================] - 51s - loss: 2.5963 - acc: 0.2389 - val_loss: 2.5923 - val_acc: 0.2271
Epoch 5/100
21000/21000 [==============================] - 51s - loss: 2.5118 - acc: 0.2417 - val_loss: 2.4105 - val_acc: 0.2486
Epoch 6/100
21000/21000 [==============================] - 51s - loss: 2.4296 - acc: 0.2486 - val_loss: 2.2909 - val_acc: 0.2613
Epoch 7/100
21000/21000 [==============================] - 51s - loss: 2.3289 - acc: 0.2565 - val_loss: 2.3633 - val_acc: 0.2548
Epoch 8/100
21000/21000 [=======================

In [71]:
model.evaluate(x=test_x, y=test_y, batch_size=500)

7389/7389 [==============================] - 5s     


[0.094075113908010988, 0.77628907119651858]

In [117]:
tests = tokenize(np.array(['(+ 1 1)',
                           '((λ) (x) (+ x 1)) 1)',
                           '(/ 1 2)',
                           '(* 2 (* 2 2))',
                           '(* 2 2)',
                           '(- 1 2)',
                           '(* 1 (+ 2 1))',
                           '(* 2 (+ 1 1))']))

model.predict(tests)

array([[ 2.18065166],
       [ 1.73664606],
       [ 0.54599768],
       [ 7.64976978],
       [ 3.83102441],
       [ 0.77701008],
       [ 2.84936213],
       [ 3.4980514 ]], dtype=float32)